In [1]:
import tensorflow as tf

In [2]:
# Check if GPU is available and set TensorFlow to use it
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPUs available:", physical_devices)
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("TensorFlow is set to use GPU.")
    except Exception as e:
        print("Could not set memory growth:", e)
else:
    print("No GPU found. Running on CPU.")

GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is set to use GPU.


In [3]:



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn
import os

In [4]:
train_path = './dataset/train'
valid_path = './dataset/val'
test_path = './dataset/test'

breeds = [
    "Holstein_cow", "Jersey_cow", "Angus_cow", "Brahman_cow", "Hereford_cow",
    "Simmental_cow", "Limousin_cow", "Guernsey_cow", "Charolais_cow", "Ayrshire_cow"
]

from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

train_batches = ImageDataGenerator(preprocessing_function=preprocess_input)\
                .flow_from_directory(directory=train_path, target_size=(224,224), classes=breeds, batch_size = 10)
valid_batches = ImageDataGenerator(preprocessing_function=preprocess_input)\
                .flow_from_directory(directory=valid_path, target_size=(224,224), classes=breeds, batch_size = 10)
test_batches = ImageDataGenerator(preprocessing_function=preprocess_input)\
                .flow_from_directory(directory=test_path, target_size=(224,224), classes=breeds, batch_size = 10, shuffle=False)

Found 2802 images belonging to 10 classes.
Found 1069 images belonging to 10 classes.
Found 1083 images belonging to 10 classes.


In [5]:
assert train_batches.n >= 1000
assert valid_batches.n >= 300
assert test_batches.n >= 150
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 10
assert train_batches.batch_size == valid_batches.batch_size == test_batches.batch_size == 10
assert train_batches.class_indices == valid_batches.class_indices == test_batches.class_indices

In [6]:
imgs , labels = next(train_batches)
print("Shape of the image batch: ", imgs.shape)
print("Shape of the label batch: ", labels.shape)

Shape of the image batch:  (10, 224, 224, 3)
Shape of the label batch:  (10, 10)


In [7]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize = (20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr,axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    plotImages(imgs)
print(labels)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [8]:
model = Sequential([
    Conv2D(filters=32,kernel_size=(3,3),activation='relu', padding='same',input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=64, kernel_size = (3,3), activation = 'relu', padding = 'same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=10, activation='softmax'),
])

In [9]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660, compute capability 7.5


In [10]:
import time
from tensorflow.keras.applications import ResNet50, DenseNet121, MobileNetV3Large
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

In [11]:
def build_model(base_model_class, input_shape, num_classes):
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = Adam(learning_rate=0.001)  
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [12]:
# Helper function to calculate mAP
def mean_average_precision(y_true, y_pred):
    # y_true and y_pred should be numpy arrays
    # This is a simple implementation for multiclass, multi-label
    import sklearn.metrics
    return sklearn.metrics.average_precision_score(y_true, y_pred, average='macro')

input_shape = (224, 224, 3)
num_classes = len(breeds)
epochs = 50
results = {}

In [17]:

print("Training MobileNetV3Large...")
mobilenetv3_model = build_model(MobileNetV3Large, input_shape, num_classes)
start_time = time.time()
history = mobilenetv3_model.fit(
    train_batches,
    validation_data=valid_batches,
    epochs=epochs,
    verbose=2,
)
training_time = time.time() - start_time

Training MobileNetV3Large...
Epoch 1/50
281/281 - 48s - loss: 1.3077 - accuracy: 0.5807 - val_loss: 1.4628 - val_accuracy: 0.5800 - 48s/epoch - 170ms/step
Epoch 2/50
281/281 - 39s - loss: 0.6920 - accuracy: 0.7727 - val_loss: 5.0048 - val_accuracy: 0.2741 - 39s/epoch - 140ms/step
Epoch 3/50
281/281 - 39s - loss: 0.4563 - accuracy: 0.8587 - val_loss: 3.1680 - val_accuracy: 0.3882 - 39s/epoch - 140ms/step
Epoch 4/50
281/281 - 40s - loss: 0.3909 - accuracy: 0.8797 - val_loss: 4.2962 - val_accuracy: 0.4051 - 40s/epoch - 141ms/step
Epoch 5/50
281/281 - 40s - loss: 0.2613 - accuracy: 0.9204 - val_loss: 7.4182 - val_accuracy: 0.3536 - 40s/epoch - 141ms/step
Epoch 6/50
281/281 - 40s - loss: 0.3280 - accuracy: 0.8940 - val_loss: 23.7049 - val_accuracy: 0.2563 - 40s/epoch - 142ms/step
Epoch 7/50
281/281 - 40s - loss: 0.2254 - accuracy: 0.9265 - val_loss: nan - val_accuracy: 0.1506 - 40s/epoch - 142ms/step
Epoch 8/50
281/281 - 39s - loss: 0.2215 - accuracy: 0.9254 - val_loss: nan - val_accuracy: 

In [18]:
val_imgs, val_labels = next(valid_batches)
val_preds = mobilenetv3_model.predict(val_imgs)
acc = np.mean(np.argmax(val_preds, axis=1) == np.argmax(val_labels, axis=1))
mAP = mean_average_precision(val_labels, val_preds)
results['MobileNetV3Large'] = {
    'training_time_sec': training_time,
    'val_accuracy': acc,
    'val_mAP': mAP,
    'history': history,
    'val_imgs': val_imgs,
    'val_labels': val_labels,
    'val_preds': val_preds
}

1/1 [==============================] - 1s 912ms/step


c:\Users\Mansal\anaconda3\envs\Mansal\lib\site-packages\sklearn\metrics\_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\Mansal\anaconda3\envs\Mansal\lib\site-packages\sklearn\metrics\_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\Mansal\anaconda3\envs\Mansal\lib\site-packages\sklearn\metrics\_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\Mansal\anaconda3\envs\Mansal\lib\site-packages\sklearn\metrics\_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\Mansal\anaconda3\envs\Mansal\lib\site-packages\sklearn\metrics\_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [19]:
with open('mobilenetv3_results.pkl', 'wb') as file:
    pickle.dump(results, file)

INFO:tensorflow:Assets written to: ram://4581a71a-79de-4a53-b21a-3f650cf1003b/assets


INFO:tensorflow:Assets written to: ram://4581a71a-79de-4a53-b21a-3f650cf1003b/assets


In [20]:
for model_name, res in results.items():
    print(f"Model: {model_name}")
    print(f"  Training Time: {res['training_time_sec']:.2f} seconds")
    print(f"  Validation Accuracy: {res['val_accuracy']:.4f}")
    print(f"  Validation mAP: {res['val_mAP']:.4f}")
    print("-" * 40)

Model: MobileNetV3Large
  Training Time: 3732.79 seconds
  Validation Accuracy: 0.7000
  Validation mAP: 0.4250
----------------------------------------
